## Brief Description of the Pedigree Module

In [1]:
using DataFrames  

### Pedigree node type:
Data for pedigree member will be stored in a pedigree node. The "seqIDs" PedNodes will be assigend integer values going from 1 to n, the number of pedigree members, such that ancestors will have lower IDs than descendents. The string ID af the parents will be stored in "sire" and "dam". If the individual is a founder, "sire" and "dam" will contain "0" and "0". The inbreeding coefficient of an individual is stored in "f".     

In [2]:
type PedNode
    seqID::Int64                                              
    sire::UTF8String
    dam::UTF8String
    f::Float64                           
end

### The Pedigree type:
The pedigree nodes will be stored in "idMap", which is an associative array that can be used to store and retrieve key-values pairs. An associative array in Julia has type "Dict". The key in idMap is the stringID of the individual, and the data associated with the key will be the PedNode of this pedigree member. The "currentID" is used to compute the sequential IDs, and, as described later, "aij" is a sparse matrix that is used to store relationship coefficients used to compute inbreeding. 

In [3]:
type Pedigree                             
    currentID::Int64                            
    idMap::Dict{UTF8String,PedNode}                               
    aij::SparseMatrixCSC{Float64,Int64}                          
end

### Read the pedigree file into a DataFrame object

In [4]:
pedFile = "ex1.ped"
df = readtable(pedFile,eltypes=[UTF8String,UTF8String,UTF8String],separator = ' ',header=false)  

,x1,x2,x3
1,AA93,A1,AA92
2,A1,C2,223G
3,FFG1,C2,C99
4,AA92,A1,FFG1


### Initialize a Pedigree object

In [11]:
idMap = Dict{UTF8String,PedNode}() # this is an empty "Dict" with key type UTF8String and value type PedNode
aij = spzeros(1,1)                 # this creates an empty 1x1 sparse matrix
ped = Pedigree(1,idMap,aij)        # here we get the Pedigree object that we will call ped

Pedigree(1,Dict{UTF8String,PedNode}(),1x1 sparse matrix with 0 Float64 entries:)

#### Make entries for sires in the idMap Dict
The PedNode value will be set to (0,"0","0",-1.0) for each sire

In [13]:
n = size(df,1) 
for i in df[:,2]                       
    if i!="0" && !haskey(ped.idMap,i)          # skip 0 and if already done
        ped.idMap[i]=PedNode(0,"0","0",-1.0)
    end
end

In [14]:
ped.idMap

Dict{UTF8String,PedNode} with 2 entries:
  "C2" => PedNode(0,"0","0",-1.0)
  "A1" => PedNode(0,"0","0",-1.0)

#### Make entries for dams in the idMap Dict
    The PedNode value will be set to (0,"0","0",-1.0) for each dam

In [16]:
    for i in df[:,3]                      
        if i!="0" && !haskey(ped.idMap,i)         # make an entry for all dams
            ped.idMap[i]=PedNode(0,"0","0",-1.0)
        end
    end 

In [17]:
ped.idMap

Dict{UTF8String,PedNode} with 6 entries:
  "FFG1" => PedNode(0,"0","0",-1.0)
  "C2"   => PedNode(0,"0","0",-1.0)
  "AA92" => PedNode(0,"0","0",-1.0)
  "C99"  => PedNode(0,"0","0",-1.0)
  "A1"   => PedNode(0,"0","0",-1.0)
  "223G" => PedNode(0,"0","0",-1.0)

#### Make entries for individuals in the pedigree and fill in sire and dam strings of PedNode

In [19]:
j = 1
for i in df[:,1]                              
    ped.idMap[i]=PedNode(0,df[j,2],df[j,3],-1.0) 
    j+=1
end 
ped.idMap

Dict{UTF8String,PedNode} with 7 entries:
  "FFG1" => PedNode(0,"C2","C99",-1.0)
  "C2"   => PedNode(0,"0","0",-1.0)
  "AA92" => PedNode(0,"A1","FFG1",-1.0)
  "C99"  => PedNode(0,"0","0",-1.0)
  "AA93" => PedNode(0,"A1","AA92",-1.0)
  "A1"   => PedNode(0,"C2","223G",-1.0)
  "223G" => PedNode(0,"0","0",-1.0)

#### Can use enumerate

In [20]:
for (j,i) in enumerate(df[:,1])                              
    ped.idMap[i]=PedNode(0,df[j,2],df[j,3],-1.0) 
end 
ped.idMap

Dict{UTF8String,PedNode} with 7 entries:
  "FFG1" => PedNode(0,"C2","C99",-1.0)
  "C2"   => PedNode(0,"0","0",-1.0)
  "AA92" => PedNode(0,"A1","FFG1",-1.0)
  "C99"  => PedNode(0,"0","0",-1.0)
  "AA93" => PedNode(0,"A1","AA92",-1.0)
  "A1"   => PedNode(0,"C2","223G",-1.0)
  "223G" => PedNode(0,"0","0",-1.0)

### Recursive function to assign sequential IDS

In [21]:
# The idea for this function came from a perl script by Bernt Guldbrandtsen
function code!(ped::Pedigree,id::UTF8String)   
    if ped.idMap[id].seqID!=0  # already done?           
        return                 # nothing more to do
    end
    sireID = ped.idMap[id].sire  
    damID  = ped.idMap[id].dam
    if sireID!="0" && ped.idMap[sireID].seqID==0 # does the sire have a sequential id?      
        code!(ped,sireID)                        # if not call the same function for sire                          
    end 
    if damID!="0" && ped.idMap[damID].seqID==0   # does the dam have a sequential id     
        code!(ped,damID)
    end
    ped.idMap[id].seqID = ped.currentID          # assign ped.current to individual and increment it 
    ped.currentID += 1
end

code! (generic function with 1 method)

In [22]:
for id in keys(ped.idMap)
    code!(ped,id)
end

In [23]:
ped.idMap

Dict{UTF8String,PedNode} with 7 entries:
  "FFG1" => PedNode(3,"C2","C99",-1.0)
  "C2"   => PedNode(1,"0","0",-1.0)
  "AA92" => PedNode(6,"A1","FFG1",-1.0)
  "C99"  => PedNode(2,"0","0",-1.0)
  "AA93" => PedNode(7,"A1","AA92",-1.0)
  "A1"   => PedNode(5,"C2","223G",-1.0)
  "223G" => PedNode(4,"0","0",-1.0)

In [28]:
n = length(ped.idMap)
a = Array(String,n,3)
f = Array(Float64,n)
for i in keys(ped.idMap)
    indSeqID  = ped.idMap[i].seqID

    a[indSeqID,1] = i
    a[indSeqID,2] = ped.idMap[i].sire
    a[indSeqID,3] = ped.idMap[i].dam
    f[indSeqID]   = ped.idMap[i].f
end
for i=1:7
    @printf("%4d %10s %10s %10s %6.3f \n", i, a[i,1], a[i,2], a[i,3], f[i])
end

   1         C2          0          0 -1.000 
   2        C99          0          0 -1.000 
   3       FFG1         C2        C99 -1.000 
   4       223G          0          0 -1.000 
   5         A1         C2       223G -1.000 
   6       AA92         A1       FFG1 -1.000 
   7       AA93         A1       AA92 -1.000 


In [31]:
X = randn(1000,2)
XPX = X'X
eig(XPX)

([954.006,1007.57],
2x2 Array{Float64,2}:
 -0.907586  -0.419865
 -0.419865   0.907586)

In [34]:
Y = [X X+randn(1000,2)*0.1]
YPY = Y'Y
eig(YPY)

([4.72027,4.84442,1905.47,2021.57],
4x4 Array{Float64,2}:
  0.669301  -0.23306   -0.62328   -0.330516
  0.230322   0.669353  -0.333222   0.622802
 -0.667341   0.228865  -0.625236  -0.333703
 -0.231613  -0.667286  -0.331006   0.625715)

In [35]:
Y = [X X+randn(1000,2)*0.01]
YPY = Y'Y
eig(YPY)

([0.0462672,0.0522468,1908.18,2015.4],
4x4 Array{Float64,2}:
  0.456596   0.53993   -0.64254   -0.295191
 -0.53996    0.456696  -0.295102   0.642485
 -0.456556  -0.539956  -0.642614  -0.295046
  0.539926  -0.456455  -0.295135   0.642669)

In [38]:
Y = [X X+randn(1000,2)*0.00]
YPY = Y'Y
eig(YPY)

([2.22643e-13,1.0291e-12,1908.01,2015.14],
4x4 Array{Float64,2}:
  0.70503    -0.0541556  -0.641761  -0.29689 
 -0.0541556  -0.70503    -0.29689    0.641761
 -0.70503     0.0541556  -0.641761  -0.29689 
  0.0541556   0.70503    -0.29689    0.641761)

In [37]:
Y = [X X+randn(1000,2)*0.001 randn(1000,2)]
YPY = Y'Y
eig(YPY)

([0.000496732,0.000574683,1036.29,1058.63,1912.52,2028.59],
6x6 Array{Float64,2}:
  0.518996     0.480276     0.00266365   0.0775284  -0.622393   -0.326477 
  0.480268    -0.518991    -0.0360387   -0.0461698  -0.331958    0.621582 
 -0.518989    -0.480238     0.00271082   0.0775105  -0.622432   -0.326474 
 -0.480245     0.518995    -0.0360498   -0.0461985  -0.331946    0.621601 
  2.9837e-5    1.21308e-5   0.532836    -0.840859   -0.0670735  -0.0673979
  4.76096e-6   2.60706e-5   0.844673     0.525995    0.0179411   0.0976427)

In [56]:
Y = [X[:,1] (X[:,1]+2*X[:,2])]
YPY = Y'Y
eig(YPY)

([756.717,5081.05],
2x2 Array{Float64,2}:
 -0.975804  0.218647
  0.218647  0.975804)

In [57]:
Y = [X[:,1] X[:,2]*10]
YPY = Y'Y
eig(YPY)

([963.027,99813.2],
2x2 Array{Float64,2}:
 -0.999998   -0.0020649
 -0.0020649   0.999998 )

In [53]:
Y = [X[:,2]+randn(1000,1)*0.01 X[:,1] (X[:,1]+2*X[:,2])]
YPY = Y'Y
eig(YPY)

([0.0728293,985.709,5850.32],
3x3 Array{Float64,2}:
 -0.816533   0.442255   0.371058
 -0.408119  -0.89681    0.170797
  0.408304  -0.0119742  0.912768)